In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

C:\Users\r2com\AppData\Local\Temp\ipykernel_8764\686016230.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('data_file.csv')

In [19]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


### 1. 결측치, 이상치 검토하기

In [26]:
sum(df.isnull().sum())

0

In [39]:
def find_outlier_rows_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    outlier_condition = (df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))
    outlier_rows = df[outlier_condition]
    return outlier_rows


# std가 큰 하나의 샘플 열 사용
outlier_rows = find_outlier_rows_iqr(df, 'MonthlyIncome')
print(outlier_rows)

      Age Attrition     BusinessTravel  DailyRate              Department  \
25     53        No      Travel_Rarely       1282  Research & Development   
29     46        No      Travel_Rarely        705                   Sales   
45     41       Yes      Travel_Rarely       1360  Research & Development   
62     50        No      Travel_Rarely        989  Research & Development   
105    59        No         Non-Travel       1420         Human Resources   
106    50        No  Travel_Frequently       1115  Research & Development   
112    54        No         Non-Travel        142         Human Resources   
119    43        No  Travel_Frequently        394                   Sales   
123    51        No      Travel_Rarely        684  Research & Development   
147    41        No  Travel_Frequently        857  Research & Development   
165    50        No      Travel_Rarely       1452  Research & Development   
186    40        No      Travel_Rarely        989  Research & Development   

### 2. 유의미한 시각화 5개 이상 수행하기

In [9]:
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')


def draw_histogram(df, x, color):
    fig = px.histogram(df, x=x, color=color)
    fig.update_layout(autosize=False, width=500)
    fig.show()

def draw_boxplot(df, x, y):
    fig = px.box(df, x=x, y=y)
    fig.update_layout(autosize=False, width=500)
    fig.show()   


draw_histogram(df, "Department", "Attrition")
draw_histogram(df, "Age", "Attrition")
draw_histogram(df, "Education", "Attrition")
draw_histogram(df, "JobSatisfaction", "Attrition")
draw_boxplot(df, "Attrition", "MonthlyIncome")


### 3. 수치형 변수 간 상관관계 파악하기

In [20]:
import plotly.figure_factory as ff

numerical_df = df.select_dtypes(include=[np.number])

corr = numerical_df.corr()

fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    annotation_text=corr.round(2).values,
    showscale=True,
    colorscale='Viridis'
)

fig.update_layout(width=1500, height=1500)
fig.show()


### 4. 파생변수 생성하기

In [30]:
# 일반적으로 하루 근무 시간을 8시간으로 가정하여 평균 시급 계산
df['AvgHourlyRate'] = df['DailyRate'] / 8